In [16]:
import caiman as cm
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import cnmf as cnmf
from caiman.source_extraction.cnmf import params as params
import matplotlib.pyplot as plt
import numpy as np

In [3]:
movie = cm.load('../data/Sue_2x_3000_40_-46.tif')

In [8]:
movie.play(plot_text=True, fr=0.5) 

In [10]:
MotionCorrect?

Init signature:
MotionCorrect(
    fname,
    min_mov=None,
    dview=None,
    max_shifts=(6, 6),
    niter_rig=1,
    splits_rig=14,
    num_splits_to_process_rig=None,
    strides=(96, 96),
    overlaps=(32, 32),
    splits_els=14,
    num_splits_to_process_els=None,
    upsample_factor_grid=4,
    max_deviation_rigid=3,
    shifts_opencv=True,
    nonneg_movie=True,
    gSig_filt=None,
    use_cuda=False,
    border_nan=True,
    pw_rigid=False,
    num_frames_split=80,
    var_name_hdf5='mov',
    is3D=False,
    indices=(slice(None, None, None), slice(None, None, None)),
)
Docstring:      class implementing motion correction operations
Init docstring:
 Constructor class for motion correction operations

 Args:
    fname: str
        path to file to motion correct

    min_mov: int16 or float32
        estimated minimum value of the movie to produce an output that is positive

    dview: ipyparallel view object list
        to perform parallel computing, if NOne will operate in si

In [14]:
mc = MotionCorrect(
    movie,
    pw_rigid=True,
    strides=(48, 48),
    overlaps=(24, 24),
    max_shifts=(6,6),
    max_deviation_rigid=3,
    )
mot_corr = mc.motion_correct(save_movie=True)
movie_corr = cm.load(mc.mmap_file)

c:\Users\sangeetha\anaconda3\envs\caiman\lib\site-packages\caiman\motion_correction.py:3118: RuntimeWarning: Mean of empty slice
  new_temp = np.nanmean(mc, 0)
100%|██████████| 1/1 [00:00<00:00,  4.59it/s]


In [15]:
movie_corr.play()

Source extraction

In [17]:
cnmf.CNMF?

Init signature:
cnmf.CNMF(
    n_processes,
    k=5,
    gSig=[4, 4],
    gSiz=None,
    merge_thresh=0.8,
    p=2,
    dview=None,
    Ain=None,
    Cin=None,
    b_in=None,
    f_in=None,
    do_merge=True,
    ssub=2,
    tsub=2,
    p_ssub=1,
    p_tsub=1,
    method_init='greedy_roi',
    alpha_snmf=0.5,
    rf=None,
    stride=None,
    memory_fact=1,
    gnb=1,
    nb_patch=1,
    only_init_patch=False,
    method_deconvolution='oasis',
    n_pixels_per_process=4000,
    block_size_temp=5000,
    num_blocks_per_run_temp=20,
    block_size_spat=5000,
    num_blocks_per_run_spat=20,
    check_nan=True,
    skip_refinement=False,
    normalize_init=True,
    options_local_NMF=None,
    minibatch_shape=100,
    minibatch_suff_stat=3,
    update_num_comps=True,
    rval_thr=0.9,
    thresh_fitness_delta=-20,
    thresh_fitness_raw=None,
    thresh_overlap=0.5,
    max_comp_update_shape=inf,
    num_times_comp_updated=inf,
    batch_update_suff_stat=False,
    s_min=None,
    remove_v

In [41]:
# general dataset-dependent parameters
fr = 30                   # imaging rate in frames per second
decay_time = 0.4          # length of a typical transient in seconds
dxy = (2., 2.)              # spatial resolution in x and y in (um per pixel)

# motion correction parameters
strides = (48, 48)          # start a new patch for pw-rigid motion correction every x pixels
overlaps = (24, 24)         # overlap between patches (width of patch = strides+overlaps)
max_shifts = (6,6)          # maximum allowed rigid shifts (in pixels)
max_deviation_rigid = 3     # maximum shifts deviation allowed for patch with respect to rigid shifts
pw_rigid = True             # flag for performing non-rigid motion correction

# CNMF parameters for source extraction and deconvolution
p = 1                       # order of the autoregressive system (set p=2 if there is visible rise time in data)
gnb = 2                     # number of global background components (set to 1 or 2)
merge_thr = 0.85            # merging threshold, max correlation allowed
bas_nonneg = False          # enforce nonnegativity constraint on calcium traces (technically on baseline)
rf = 15                     # half-size of the patches in pixels (patch width is rf*2 + 1)
stride_cnmf = 6             # amount of overlap between the patches in pixels (overlap is stride_cnmf+1)
K = 4                       # number of components per patch
gSig = np.array([4, 4])     # expected half-width of neurons in pixels
gSiz = 2*gSig + 1           # half-width of bounding box created around neurons during initialization
method_init = 'greedy_roi'  # initialization method (if analyzing dendritic data see demo_dendritic.ipynb)
ssub = 1                    # spatial subsampling during initialization
tsub = 1                    # temporal subsampling during intialization

# parameters for component evaluation
min_SNR = 2.0               # signal to noise ratio for accepting a component
rval_thr = 0.85             # space correlation threshold for accepting a component
cnn_thr = 0.99              # threshold for CNN based classifier
cnn_lowest = 0.1            # neurons with cnn probability lower than this value are rejected


In [44]:
parameter_dict = {'fnames': "C:\\Users\\sangeetha\\Courses\\Intro-To-Calcium-Imaging-Analysis\\data\\Sue_2x_3000_40_-46.tif",
                  'fr': fr,
                  'dxy': dxy,
                  'decay_time': decay_time,
                  'strides': strides,
                  'overlaps': overlaps,
                  'max_shifts': max_shifts,
                  'max_deviation_rigid': max_deviation_rigid,
                  'pw_rigid': pw_rigid,
                  'p': p,
                  'nb': gnb,
                  'rf': rf,
                  'K': K,
                  'gSig': gSig,
                  'gSiz': gSiz,
                  'stride': stride_cnmf,
                  'method_init': method_init,
                  'rolling_sum': True,
                  'only_init': True,
                  'ssub': ssub,
                  'tsub': tsub,
                  'merge_thr': merge_thr,
                  'bas_nonneg': bas_nonneg,
                  'min_SNR': min_SNR,
                  'rval_thr': rval_thr,
                  'use_cnn': True,
                  'min_cnn_thr': cnn_thr,
                  'cnn_lowest': cnn_lowest}

parameters = params.CNMFParams(params_dict=parameter_dict) # CNMFParams is the parameters class

In [43]:
parameters = params.CNMFParams(params_dict={'fnames':fnames})
cnmf_model = cnmf.CNMF(
    n_processes=1,
    )
cnmf_model.fit_file(motion_correct=False)

TypeError: 'NoneType' object is not subscriptable